In [1]:
import numpy as np
import itertools

In [2]:
a = np.array([0,0,0,1,2], dtype=np.float)
b = np.array([0,1,0,1,1], dtype=np.float)
np.divide(a, a+b, out=np.ones_like(a), where=(a+b)!=0)

array([1.        , 0.        , 1.        , 0.5       , 0.66666667])

In [3]:
np.array([1,2,4,3,1]).argsort()[::-1][2]

1

In [4]:
np.array([[1,2,4,3,1],[2,4,1,3,5]]).argsort()[:,::-1][:,2]

array([1, 3])

In [25]:
result = np.array([
    [[3,1,1],
     [2,2,1],
     [3,2,1],
     [2,5,1]],
    [[2,1,1],
     [6,2,1],
     [1,0,1],
     [1,1,1]]
], dtype=np.float)

In [26]:
num_players = result.shape[0]
num_arms = result.shape[1]
# Experienced average for each player. 1 if each machine has never been pulled.
experienced_average = np.divide(result[:,:,0], result[:,:,0]+result[:,:,1], out=np.ones_like(result[:,:,0]), where=(result[:,:,0]+result[:,:,1])!=0)
beta_list = np.pi * (result[:,:,0]+result[:,:,1]+2) * np.sqrt((result[:,:,0]+result[:,:,1]+3)/(6*(result[:,:,0]+1)*(result[:,:,1]+1)))
# Optimal beta for each player.
beta = beta_list[range(num_players), experienced_average.argsort()[:,::-1][:,num_players]]
preference = experienced_average * beta.reshape(num_players, 1)
preference -= preference.max(axis=1).reshape(-1,1)
preference = np.exp(preference)
print(beta)
print(preference)

[6.78661579 6.41274915]
[[1.         0.18329582 0.36131961 0.04281235]
 [0.11793955 0.20125404 1.         0.04050319]]


In [44]:
selection_prob = np.zeros((num_players, num_arms))
all_permutations = list(itertools.permutations(range(num_arms)))
for order in all_permutations:
    tmp_pref = preference.copy()
    tmp_pref[:, order[0]] = 0
    p = preference[:, order[0]]/preference.sum(axis=1) * preference[:, order[1]]/tmp_pref.sum(axis=1)
    selection_prob[:, order[0]] += p/2
    selection_prob[:, order[1]] += p/2

In [45]:
print(selection_prob)
selection_prob /= selection_prob.sum(axis=1).reshape(-1,1)
selection_prob

[[0.91528301 0.34925848 0.65110922 0.0843493 ]
 [0.34561831 0.57811279 0.95566088 0.12060802]]


array([[0.4576415 , 0.17462924, 0.32555461, 0.04217465],
       [0.17280915, 0.28905639, 0.47783044, 0.06030401]])

In [50]:
selections = list(itertools.product(range(num_arms), repeat=num_players))
selection_matrix = selection_prob[range(num_players),selections].prod(1)
selection_matrix = selection_matrix.reshape(num_arms, num_arms)
selection_matrix /= selection_matrix.sum()
print(selection_matrix)

[[0.07908464 0.1322842  0.21867504 0.02759762]
 [0.03017753 0.0504777  0.08344317 0.01053084]
 [0.05625882 0.09410364 0.1555599  0.01963225]
 [0.00728817 0.01219085 0.02015233 0.0025433 ]]


In [47]:
input_state = np.array([0, 1/12, 1/12, 1/12, 1/12, 0, 1/12, 1/12, 1/12, 1/12, 0, 1/12, 1/12, 1/12, 1/12, 0])

In [54]:
joint_matrix = selection_matrix * input_state.reshape(num_arms, num_arms)
joint_matrix = (joint_matrix/joint_matrix.sum()).reshape(-1,)

In [55]:
joint_matrix

array([0.        , 0.18570519, 0.30698366, 0.0387425 , 0.04236427,
       0.        , 0.11714043, 0.01478357, 0.07897809, 0.13210598,
       0.        , 0.02756044, 0.01023138, 0.01711394, 0.02829055,
       0.        ])